In [1]:
import json
import numpy as np
import random
import nltk
import utils as u
#nltk.download('punkt')
#nltk.download('wordnet')

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD


class ChatModel:

    def __init__(self):
        #Call tokenizing procedure
        w, words, documents, classes, self._intents = self.tokenizing('intents.json')

        #Call lemmatizing procedure
        w, words, documents, classes, lemmatizer = self.lemmatizing(w, words, documents, classes)

        #Call training_data procedure
        self._train_x, self._train_y = self.training_data(w, words, documents, classes, lemmatizer)

        #Call tokenizing procedure
        self._model = self.training(self._train_x, self._train_y)
    

    def tokenizing(self,url):
        words=[]
        classes = []
        documents = []
        intents = json.loads(open(url).read())

        for intent in intents['intents']:
            for pattern in intent['patterns']:
                #tokenize each word
                w = nltk.word_tokenize(pattern)
                words.extend(w)
                #add documents in the corpus
                documents.append((w, intent['tag']))
                # add to our classes list
                if intent['tag'] not in classes:
                    classes.append(intent['tag'])

        return w, words, documents, classes, intents

    def lemmatizing(self, w, words, documents, classes):
        ignore_words = ['?', '!']
        lemmatizer = nltk.stem.WordNetLemmatizer()

        # lemmatize, lower each word and remove duplicates
        words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

        # sort classes and words
        classes = sorted(list(set(classes)))
        words = sorted(list(set(words)))
        # documents = combination between patterns and intents
        print (len(documents), "documents")

        # classes = intents
        print (len(classes), "classes", classes)

        # words = all words, vocabulary
        print (len(words), "unique lemmatized words", words)

        u.create_pickle(words, 'pickles\words.pkl') 
        u.create_pickle(classes, 'pickles\classes.pkl')
        return w, words, documents, classes, lemmatizer

    def training_data(self, w, words, documents, classes, lemmatizer):
        # create our training data
        training = []
        train_x = []
        train_y = []
        # create an empty array for our output
        output_empty = [0] * len(classes)

        # training set, bag of words for each sentence
        for doc in documents:
            # initialize our bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # lemmatize each word - create base word, in attempt to represent related words
            pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
            # create our bag of words array with 1, if word match found in current pattern

            for w in words:
                bag.append(1) if w in pattern_words else bag.append(0)

            # output is a '0' for each tag and '1' for current tag (for each pattern)
            output_row = list(output_empty)
            output_row[classes.index(doc[1])] = 1
            training.append([bag, output_row])

        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)
        # create train and test lists. X - patterns, Y - intents
        train_x = list(training[:,0])
        train_y = list(training[:,1])

        print("Training data created")
        return train_x, train_y

    def training(self,train_x, train_y):
        #Sequential from Keras
        # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
        # equal to number of intents to predict output intent with softmax
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        #fitting and saving the model 
        hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
        model.save('chatbot_model.h5', hist)
        print("modeseql created")

        return model

    def get_train_x(self):
        return self._train_x

    def get_train_y(self):
        return self._train_y
    
    def get_model(self):
        return self._model

    def get_intents(self):
        return self._intents
        

2023-08-04 00:10:23.161413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class ChatApp:

    def __init__(self):
        self.cM = chatModel()
        self._lemmatizer = nltk.stem.WordNetLemmatizer()
        self._model = load_model('chatbot_model.h5')
        self._intents = self.cM.get_intents() 
        self._words = u.load_pickle('pickles\words.pkl')
        self._classes = u.load_pickle('pickles\classes.pkl')
        self._tags_=[]
        
    def clean_up_sentence(self,sentence):
        # tokenize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [self._lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words

    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(self, sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words) 
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))

    def predict_class(self, sentence, model):
        ERROR_THRESHOLD = 0.05
        # filter out predictions below a threshold
        p = self.bow(sentence, self._words, show_details=False)
        res = self._model.predict(np.array([p]))[0]
        
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self._classes[r[0]], "probability": str(r[1])})
        return return_list


    def getResponse(self, ints, intents_json):
        tag = ints[0]['intent']
        context=""
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag and i['context']==context) or (i['tag']==tag and i['context']==""):
                result = random.choice(i['responses'])
                self._tags_.append(tag)
                break
        return result

    def chatbot_response(self, text):
        ints = self.predict_class(text, self._model)
        print(ints)
        res = self.getResponse(ints, self._intents)
        return res

In [3]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
from chatapp import ChatApp as cA

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = cA().chatbot_response(msg)
            ChatLog.insert(END, "Fern: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)


base = Tk()
base.title("FERN")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

greeting="Hello, my name is Fern and I will be assisting you today. I am a chatbot that aims to help victims of domestic violence by offering solutions and recommending lawyers. I will begin with asking you a range of questions to better understand your current situation. Based on the information you have provided, I will give some recommendations on how you can move forward. In case you would like further assistance by a lawyer, we will also provide a list of lawyers that we think are best suited for your needs. Before we delve in, please know that the more details you can provide us, the better advice we will be able to give you. These small details can be very important from a legal perspective so please be truthful so we can best assess your legal position. We understand that this is a sensitive matter so please take as much time as you need and ensure that you are in a safe environment to talk to us. If you are living with the person who is inflicting harm on you, you can click the red button at the top right to disguise this website as an online shopping site. Does everything make sense so far? "
ChatLog.config(state=NORMAL)
ChatLog.insert(END, "Fern: " + greeting + '\n\n')
ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

base.mainloop()

15 documents
7 classes ['client_conflict', 'family', 'last_diagnostics', 'permissions', 'physical', 'secondary_diagnostics', 'threat_level_diagnostics']
117 unique lemmatized words [',', '.', 'a', 'about', 'abusive', 'aggressive', 'and', 'at', 'bad', 'be', 'because', 'before', 'begin', 'beginning', 'but', 'called', 'can', 'close', 'continue', 'do', 'don', 'family', 'for', 'friend', 'get', 'hand', 'happens', 'have', 'health', 'her', 'honest', 'i', 'in', 'is', 'it', 'just', 'kicked', 'kicking', 'kid', 'knife', 'like', 'll', 'loss', 'm', 'make', 'marriage', 'may', 'me', 'mental', 'my', 'neighbour', 'never', 'not', 'now', 'off', 'often', 'ok', 'okay', 'on', 'once', 'only', 'our', 'partner', 'physical', 'physically', 'place', 'police', 'pretty', 'punching', 'pushing', 'regularly', 'safe', 'screaming', 'sense', 'set', 'she', 'shouting', 'so', 'something', 'sometimes', 'stab', 'start', 'started', 'stuff', 'sure', 't', 'taking', 'tell', 'the', 'there', 'they', 'think', 'this', 'throwing', 'tim

/Users/adamchoong/Desktop/MAC x LSS legal hackathon/retrival based chatbot /chat_model.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
2023-08-04 00:10:33.487705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 0s 1ms/step - loss: 2.0719 - accuracy: 0.2000
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 1.8235 - accuracy: 0.2000
Epoch 3/200
3/3 [==============================] - 0s 981us/step - loss: 1.8417 - accuracy: 0.2667
Epoch 4/200
3/3 [==============================] - 0s 846us/step - loss: 1.6021 - accuracy: 0.6000
Epoch 5/200
3/3 [==============================] - 0s 908us/step - loss: 1.8343 - accuracy: 0.2667
Epoch 6/200
3/3 [==============================] - 0s 962us/step - loss: 1.5256 - accuracy: 0.5333
Epoch 7/200
3/3 [==============================] - 0s 884us/step - loss: 1.4897 - accuracy: 0.4667
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 1.3188 - accuracy: 0.6667
Epoch 9/200
3/3 [==============================] - 0s 887us/step - loss: 1.2131 - accuracy: 0.6667
Epoch 10/200
3/3 [==============================] - 0s 830us/step - loss: 1.2516 - accuracy: 0.6667
Epoch 11/200
3/3 [=========

3/3 [==============================] - 0s 803us/step - loss: 0.0333 - accuracy: 1.0000
Epoch 84/200
3/3 [==============================] - 0s 1ms/step - loss: 0.0543 - accuracy: 1.0000
Epoch 85/200
3/3 [==============================] - 0s 808us/step - loss: 0.0390 - accuracy: 1.0000
Epoch 86/200
3/3 [==============================] - 0s 875us/step - loss: 0.1514 - accuracy: 0.9333
Epoch 87/200
3/3 [==============================] - 0s 815us/step - loss: 0.0556 - accuracy: 1.0000
Epoch 88/200
3/3 [==============================] - 0s 914us/step - loss: 0.1445 - accuracy: 0.9333
Epoch 89/200
3/3 [==============================] - 0s 884us/step - loss: 0.0435 - accuracy: 1.0000
Epoch 90/200
3/3 [==============================] - 0s 769us/step - loss: 0.0442 - accuracy: 1.0000
Epoch 91/200
3/3 [==============================] - 0s 862us/step - loss: 0.0963 - accuracy: 1.0000
Epoch 92/200
3/3 [==============================] - 0s 911us/step - loss: 0.0272 - accuracy: 1.0000
Epoch 93/200
3/

3/3 [==============================] - 0s 758us/step - loss: 9.9454e-04 - accuracy: 1.0000
Epoch 165/200
3/3 [==============================] - 0s 872us/step - loss: 0.0678 - accuracy: 1.0000
Epoch 166/200
3/3 [==============================] - 0s 958us/step - loss: 0.0910 - accuracy: 1.0000
Epoch 167/200
3/3 [==============================] - 0s 774us/step - loss: 0.0642 - accuracy: 1.0000
Epoch 168/200
3/3 [==============================] - 0s 865us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 169/200
3/3 [==============================] - 0s 723us/step - loss: 0.0213 - accuracy: 1.0000
Epoch 170/200
3/3 [==============================] - 0s 777us/step - loss: 0.0507 - accuracy: 1.0000
Epoch 171/200
3/3 [==============================] - 0s 820us/step - loss: 0.0191 - accuracy: 1.0000
Epoch 172/200
3/3 [==============================] - 0s 920us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 173/200
3/3 [==============================] - 0s 856us/step - loss: 0.0240 - accuracy: 1.0000


...

3/3 [==============================] - 0s 821us/step - loss: 0.1268 - accuracy: 0.9333
Epoch 46/200
3/3 [==============================] - 0s 841us/step - loss: 0.2468 - accuracy: 1.0000
Epoch 47/200
3/3 [==============================] - 0s 811us/step - loss: 0.2148 - accuracy: 1.0000
Epoch 48/200
3/3 [==============================] - 0s 762us/step - loss: 0.0555 - accuracy: 1.0000
Epoch 49/200
3/3 [==============================] - 0s 814us/step - loss: 0.0535 - accuracy: 1.0000
Epoch 50/200
3/3 [==============================] - 0s 766us/step - loss: 0.1855 - accuracy: 0.9333
Epoch 51/200
3/3 [==============================] - 0s 794us/step - loss: 0.1510 - accuracy: 1.0000
Epoch 52/200
3/3 [==============================] - 0s 732us/step - loss: 0.0813 - accuracy: 1.0000
Epoch 53/200
3/3 [==============================] - 0s 860us/step - loss: 0.2253 - accuracy: 0.9333
Epoch 54/200
3/3 [==============================] - 0s 750us/step - loss: 0.0721 - accuracy: 1.0000
Epoch 55/200


3/3 [==============================] - 0s 889us/step - loss: 0.0130 - accuracy: 1.0000
Epoch 127/200
3/3 [==============================] - 0s 816us/step - loss: 0.1255 - accuracy: 0.9333
Epoch 128/200
3/3 [==============================] - 0s 805us/step - loss: 0.0458 - accuracy: 1.0000
Epoch 129/200
3/3 [==============================] - 0s 768us/step - loss: 0.0133 - accuracy: 1.0000
Epoch 130/200
3/3 [==============================] - 0s 805us/step - loss: 0.0617 - accuracy: 1.0000
Epoch 131/200
3/3 [==============================] - 0s 778us/step - loss: 0.0478 - accuracy: 1.0000
Epoch 132/200
3/3 [==============================] - 0s 839us/step - loss: 0.0609 - accuracy: 1.0000
Epoch 133/200
3/3 [==============================] - 0s 826us/step - loss: 0.0474 - accuracy: 1.0000
Epoch 134/200
3/3 [==============================] - 0s 751us/step - loss: 0.1388 - accuracy: 0.9333
Epoch 135/200
3/3 [==============================] - 0s 846us/step - loss: 0.0391 - accuracy: 1.0000
Epoc

1/1 [==============================] - 0s 26ms/step
15 documents
7 classes ['client_conflict', 'family', 'last_diagnostics', 'permissions', 'physical', 'secondary_diagnostics', 'threat_level_diagnostics']
117 unique lemmatized words [',', '.', 'a', 'about', 'abusive', 'aggressive', 'and', 'at', 'bad', 'be', 'because', 'before', 'begin', 'beginning', 'but', 'called', 'can', 'close', 'continue', 'do', 'don', 'family', 'for', 'friend', 'get', 'hand', 'happens', 'have', 'health', 'her', 'honest', 'i', 'in', 'is', 'it', 'just', 'kicked', 'kicking', 'kid', 'knife', 'like', 'll', 'loss', 'm', 'make', 'marriage', 'may', 'me', 'mental', 'my', 'neighbour', 'never', 'not', 'now', 'off', 'often', 'ok', 'okay', 'on', 'once', 'only', 'our', 'partner', 'physical', 'physically', 'place', 'police', 'pretty', 'punching', 'pushing', 'regularly', 'safe', 'screaming', 'sense', 'set', 'she', 'shouting', 'so', 'something', 'sometimes', 'stab', 'start', 'started', 'stuff', 'sure', 't', 'taking', 'tell', 'the'

3/3 [==============================] - 0s 759us/step - loss: 0.0469 - accuracy: 1.0000
Epoch 71/200
3/3 [==============================] - 0s 822us/step - loss: 0.1673 - accuracy: 1.0000
Epoch 72/200
3/3 [==============================] - 0s 772us/step - loss: 0.2595 - accuracy: 0.9333
Epoch 73/200
3/3 [==============================] - 0s 874us/step - loss: 0.3206 - accuracy: 0.8667
Epoch 74/200
3/3 [==============================] - 0s 773us/step - loss: 0.0459 - accuracy: 1.0000
Epoch 75/200
3/3 [==============================] - 0s 805us/step - loss: 0.1027 - accuracy: 1.0000
Epoch 76/200
3/3 [==============================] - 0s 794us/step - loss: 0.0656 - accuracy: 1.0000
Epoch 77/200
3/3 [==============================] - 0s 755us/step - loss: 0.0361 - accuracy: 1.0000
Epoch 78/200
3/3 [==============================] - 0s 739us/step - loss: 0.0293 - accuracy: 1.0000
Epoch 79/200
3/3 [==============================] - 0s 864us/step - loss: 0.1534 - accuracy: 0.9333
Epoch 80/200


3/3 [==============================] - 0s 861us/step - loss: 0.0421 - accuracy: 1.0000
Epoch 152/200
3/3 [==============================] - 0s 849us/step - loss: 0.0268 - accuracy: 1.0000
Epoch 153/200
3/3 [==============================] - 0s 792us/step - loss: 0.0469 - accuracy: 1.0000
Epoch 154/200
3/3 [==============================] - 0s 870us/step - loss: 0.0217 - accuracy: 1.0000
Epoch 155/200
3/3 [==============================] - 0s 775us/step - loss: 0.0160 - accuracy: 1.0000
Epoch 156/200
3/3 [==============================] - 0s 792us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 157/200
3/3 [==============================] - 0s 795us/step - loss: 0.0146 - accuracy: 1.0000
Epoch 158/200
3/3 [==============================] - 0s 841us/step - loss: 0.0356 - accuracy: 1.0000
Epoch 159/200
3/3 [==============================] - 0s 872us/step - loss: 0.0165 - accuracy: 1.0000
Epoch 160/200
3/3 [==============================] - 0s 823us/step - loss: 0.0180 - accuracy: 1.0000
Epoc